In [1]:
import os
import time
from glob import glob

import rasterio as rio
from osgeo import gdal, osr
from rasterstats import zonal_stats
import fiona
import geopandas as gpd
import pandas as pd
import numpy as np

import multiprocess
from functools import partial
import itertools

home = "/share/klmarti3/kmcquil/Chapter2_mechanisms_forest_water_cycling"
home = "/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling"

def chunks(data, n):
    """Yield successive n-sized chunks from a slice-able iterable."""
    for i in range(0, len(data), n):
        yield data[i:i+n]

def zonal_stats_partial(tif, feats):
    """Wrapper for zonal stats, takes a list of features"""
    return zonal_stats(feats, tif, all_touched=True, stats='mean')

def get_file_info(files):
    row_list = []
    for file in files:
        filename = file
        basename = os.path.basename(filename)
        breakdown_path = basename.split("_")   

        Type = breakdown_path[0]
        Var = breakdown_path[1]
        if Type == "anomalies": 
            Season = breakdown_path[2]
            Year = breakdown_path[3][:-4]
        else: 
            Season = breakdown_path[2][:-4]
            Year = np.nan 

        row_list.append({'Type':Type, 'Var':Var, 'Season':Season, 'Year':Year})
    
    return pd.DataFrame(row_list)

def summarize_multithread(pool, variable):
    
    files = glob(os.path.join(home, "Data", "Climate", "tifs", "*" + variable  + "*.tif"))
    file_info = get_file_info(files)
    print(len(files))
    K = -1

    for tif in files:
        
        # track progress 
        print(tif)
        tic = time.perf_counter()
        K +=1
        
        # map the function to chunks of features across the requested cores and convert to a df 
        stats_lists = pool.map(partial(zonal_stats_partial, tif), chunks(features, cores))
        stats = pd.DataFrame(list(itertools.chain(*stats_lists)))
        stats.columns = ['Value']
        df = pd.concat([shp['NHDPlusID'], stats], axis=1)

        # add additional info for indexing 
        df['Type'] = [file_info.iloc[K].Type]* df.shape[0]
        df['Var'] = [file_info.iloc[K].Var]* df.shape[0]
        df['Season'] = [file_info.iloc[K].Season]* df.shape[0]
        df['Year'] = [file_info.iloc[K].Year]* df.shape[0]

        if K == 0:
            df_all = df
        else:
            df_all = pd.concat([df_all, df], axis=0)
        
        toc = time.perf_counter()
        print('iteration ' + str(K) + ' ran for ' + str(toc-tic) + ' seconds')

    p.close()
    p.join()
    df_all.to_csv(os.path.join(home, "Data", "Climate", "Summary", variable + ".csv"))

In [2]:
# get the shapefile into the correct crs and create a test subset 
#gpkg = gpd.read_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_sbr.gpkg"))
#crs = rio.open(glob(os.path.join(home, "Data", "Climate", "tifs", "*" + 'tmin'  + "*.tif"))[0]).crs
#gpkg_proj = gpkg.to_crs(crs)
#gpkg_proj.to_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_climate.shp"))

gpkg_proj = gpd.read_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_climate.shp"))
test = gpkg_proj[0:5]
test.to_file(os.path.join(home, "Data", "Catchments", "Headwater", "TESTING_headwater_catchments_climate.shp"))

/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


In [3]:
#shp_file = os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_sbr_shp_4269.shp")
#shp = gpd.read_file(os.path.join(home, "Data", "Catchments", "Headwater", "headwater_catchments_sbr_shp_4269.shp"))
shp_file = os.path.join(home, "Data", "Catchments", "Headwater", "TESTING_headwater_catchments_climate.shp")
shp = gpd.read_file(os.path.join(home, "Data", "Catchments", "Headwater", "TESTING_headwater_catchments_climate.shp"))

with fiona.open(shp_file) as src:
        features = list(src)
print("shapes done!")

shapes done!


In [4]:
if __name__ == "__main__":
    # Create a process pool using all cores
    cores = 2
    p = multiprocess.Pool(cores)
    summarize_multithread(p, 'tmin')

/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")


118
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/tifs/climatology_tmin_GS.tif


/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2

iteration 0 ran for 0.2523024580000026 seconds
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/tifs/climatology_tmin_notGS.tif


/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2

...

iteration 112 ran for 0.18653183300000364 seconds
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/tifs/anomalies_tmin_annual_2017.tif
iteration 113 ran for 0.1856387920000202 seconds
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/tifs/anomalies_tmin_annual_2018.tif


/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2

iteration 114 ran for 0.19937525000000278 seconds
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/tifs/anomalies_tmin_annual_2019.tif
iteration 115 ran for 0.19745479200000204 seconds
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/tifs/anomalies_tmin_annual_2020.tif


/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2-python-env/lib/python3.9/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/katiemcquillan/opt/anaconda3/envs/ch2

iteration 116 ran for 0.1861482919999844 seconds
/Volumes/GoogleDrive/My Drive/Chapter2_mechanisms_forest_water_cycling/Data/Climate/tifs/anomalies_tmin_annual_2021.tif
iteration 117 ran for 0.18621954200000346 seconds


In [15]:
p.close()